# Real Estate Investment Advisor - EDA, Modeling and Streamlit

This notebook performs exploratory data analysis (EDA), feature engineering, trains quick classification and regression models, logs experiments with MLflow, and writes a small Streamlit app for inference.

Notes: training is intentionally small/fast so it runs quickly on modest machines.

In [ ]:
# Install required libraries (runs in notebook).
!pip install -q pandas numpy matplotlib seaborn scikit-learn plotly mlflow lightgbm joblib streamlit

In [ ]:
# Imports
import warnings
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import joblib
sns.set(style='whitegrid')

In [ ]:
# Load dataset (assumes the CSV is in the same folder)
df = pd.read_csv('india_housing_prices.csv')
df.head()

In [ ]:
# Basic info and cleaning
df.info()
print('Rows, cols:', df.shape)
# Convert numeric-like columns if necessary
for col in ['Price_per_SqFt','Price_in_Lakhs','Size_in_SqFt','Public_Transport_Accessibility','Nearby_Schools','Nearby_Hospitals','Age_of_Property']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
# Trim whitespace for categorical columns
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# Quick summary statistics
df.describe(include='all').T

**1-5: Price & Size Analysis**

In [ ]:
# Distribution of property prices
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.histplot(df['Price_in_Lakhs'].dropna(), bins=30, kde=True)
plt.title('Distribution of Property Prices (Lakhs)')
plt.subplot(1,2,2)
sns.boxplot(x=df['Price_in_Lakhs'])
plt.title('Boxplot of Property Prices')
plt.tight_layout()

# Distribution of property sizes
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.histplot(df['Size_in_SqFt'].dropna(), bins=30, kde=True, color='orange')
plt.title('Distribution of Property Sizes (SqFt)')
plt.subplot(1,2,2)
sns.boxplot(x=df['Size_in_SqFt'])
plt.title('Boxplot of Property Sizes')
plt.tight_layout()

In [ ]:
# Price per sqft by property type
plt.figure(figsize=(10,5))
sns.boxplot(data=df, x='Property_Type', y='Price_per_SqFt')
plt.title('Price per SqFt by Property Type')
plt.xticks(rotation=45)
plt.show()

# Relationship between size and price
plt.figure(figsize=(6,5))
sns.scatterplot(data=df, x='Size_in_SqFt', y='Price_in_Lakhs', hue='Property_Type', alpha=0.8)
plt.title('Property Size vs Price')
plt.show()

# Outliers detection for price_per_sqft and size: show top 5 by z-score
from scipy import stats
df['ppsf_z'] = np.abs(stats.zscore(df['Price_per_SqFt'].fillna(df['Price_per_SqFt'].median())))
df['size_z'] = np.abs(stats.zscore(df['Size_in_SqFt'].fillna(df['Size_in_SqFt'].median())))
display(df.sort_values('ppsf_z', ascending=False).head(5)[['ID','Price_per_SqFt','ppsf_z','Size_in_SqFt']])
display(df.sort_values('size_z', ascending=False).head(5)[['ID','Size_in_SqFt','size_z','Price_in_Lakhs']])

**6-10: Location-based Analysis**

In [ ]:
# Average price per sqft by state
state_ppsf = df.groupby('State')['Price_per_SqFt'].mean().sort_values(ascending=False)
state_ppsf.head(20)
plt.figure(figsize=(10,5))
state_ppsf.plot(kind='bar')
plt.title('Average Price per SqFt by State')
plt.ylabel('Price per SqFt')
plt.show()

# Average property price by city
city_price = df.groupby('City')['Price_in_Lakhs'].mean().sort_values(ascending=False)
city_price.head(20)
plt.figure(figsize=(10,5))
city_price.head(20).plot(kind='bar')
plt.title('Average Property Price by City (Top 20)')
plt.ylabel('Average Price (Lakhs)')
plt.show()

# Median age of properties by locality (show top 20 localities with most records)
locality_median_age = df.groupby('Locality')['Age_of_Property'].median()
locality_counts = df['Locality'].value_counts().loc[lambda x: x>0]
popular_localities = locality_counts.head(20).index
plt.figure(figsize=(10,5))
locality_median_age.loc[popular_localities].sort_values().plot(kind='bar')
plt.title('Median Age of Properties (Popular Localities)')
plt.ylabel('Median Age')
plt.show()

# BHK distribution across cities (heatmap of counts)
bhk_city = pd.crosstab(df['City'], df['BHK'])
plt.figure(figsize=(12,6))
sns.heatmap(bhk_city, cmap='Blues', annot=False)
plt.title('BHK Distribution Across Cities')
plt.show()

# Price trends for top 5 most expensive localities by median price_per_sqft
top_localities = df.groupby('Locality')['Price_per_SqFt'].median().sort_values(ascending=False).head(5).index
plt.figure(figsize=(10,6))
for loc in top_localities:
    subset = df[df['Locality']==loc]
    plt.plot(subset['Year_Built'], subset['Price_per_SqFt'], 'o-', label=loc)
plt.legend()
plt.title('Price per SqFt vs Year Built for Top 5 Localities')
plt.xlabel('Year Built')
plt.ylabel('Price per SqFt')
plt.show()

**11-15: Feature Relationship & Correlation**

In [ ]:
# Correlation between numeric features
num_cols = ['Price_in_Lakhs','Price_per_SqFt','Size_in_SqFt','Age_of_Property','Public_Transport_Accessibility','Nearby_Schools','Nearby_Hospitals']
corr = df[num_cols].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Numeric Feature Correlations')
plt.show()

# Nearby schools vs price_per_sqft: scatter + trend
plt.figure(figsize=(6,4))
sns.boxplot(x=pd.cut(df['Nearby_Schools'].fillna(0), bins=4), y=df['Price_per_SqFt'])
plt.title('Price per SqFt by Nearby Schools (binned)')
plt.show()
# Nearby hospitals vs price_per_sqft
plt.figure(figsize=(6,4))
sns.boxplot(x=pd.cut(df['Nearby_Hospitals'].fillna(0), bins=4), y=df['Price_per_SqFt'])
plt.title('Price per SqFt by Nearby Hospitals (binned)')
plt.show()

# Price by furnished status
plt.figure(figsize=(6,4))
sns.boxplot(data=df, x='Furnished_Status', y='Price_in_Lakhs')
plt.title('Price by Furnished Status')
plt.show()

# Price per sqft by facing direction
plt.figure(figsize=(10,4))
sns.boxplot(data=df, x='Facing', y='Price_per_SqFt')
plt.title('Price per SqFt by Facing Direction')
plt.xticks(rotation=45)
plt.show()

**16-20: Investment / Amenities / Ownership Analysis**

In [ ]:
# Owner type counts and availability status
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
df['Owner_Type'].value_counts().plot(kind='bar')
plt.title('Properties by Owner Type')
plt.subplot(1,2,2)
df['Availability_Status'].value_counts().plot(kind='bar', color='orange')
plt.title('Availability Status')
plt.tight_layout()
plt.show()

# Does parking affect price? simple comparison
if 'Parking_Space' in df.columns:
    plt.figure(figsize=(6,4))
    sns.boxplot(data=df, x='Parking_Space', y='Price_in_Lakhs')
    plt.title('Price by Parking Space Availability')
    plt.show()

# Amenities effect: create amenities_count from comma separated list
def count_amenities(x):
    if pd.isna(x):
        return 0
    if isinstance(x, str):
        # remove surrounding quotes if any and split by comma
        return len([a.strip() for a in x.replace('"','').split(',') if a.strip()])
    return 0
df['amenities_count'] = df['Amenities'].apply(count_amenities) if 'Amenities' in df.columns else 0
plt.figure(figsize=(6,4))
sns.regplot(x='amenities_count', y='Price_per_SqFt', data=df, scatter_kws={'alpha':0.6})
plt.title('Price per SqFt vs Amenities Count')
plt.show()

# Public transport accessibility relation to price per sqft
plt.figure(figsize=(6,4))
sns.scatterplot(x='Public_Transport_Accessibility', y='Price_per_SqFt', data=df)
plt.title('Public Transport Accessibility vs Price per SqFt')
plt.show()

**Modeling: Quick classification (investment prediction) and regression (price forecasting)**

In [ ]:
# Prepare features for modeling (lightweight)
model_df = df.copy()
# Fill NAs with sensible defaults
model_df['Price_per_SqFt'] = model_df['Price_per_SqFt'].fillna(model_df['Price_per_SqFt'].median())
model_df['Price_in_Lakhs'] = model_df['Price_in_Lakhs'].fillna(model_df['Price_in_Lakhs'].median())
model_df['Size_in_SqFt'] = model_df['Size_in_SqFt'].fillna(model_df['Size_in_SqFt'].median())
model_df['Public_Transport_Accessibility'] = model_df['Public_Transport_Accessibility'].fillna(model_df['Public_Transport_Accessibility'].median())
model_df['Nearby_Schools'] = model_df['Nearby_Schools'].fillna(0)
model_df['Nearby_Hospitals'] = model_df['Nearby_Hospitals'].fillna(0)
model_df['amenities_count'] = model_df.get('amenities_count', 0).fillna(0)
# Simple encoding for categorical features: Property_Type, Furnished_Status, Owner_Type, Facing
cat_cols = ['Property_Type','Furnished_Status','Owner_Type','Facing']
for c in cat_cols:
    model_df[c] = model_df[c].fillna('Unknown')
# Feature engineering: amenities_count already done
# Create an investment score and binary label (quick heuristic)
# Higher public transport + more amenities + lower price_per_sqft -> higher investment potential
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
score_df = model_df[['Price_per_SqFt','Public_Transport_Accessibility','amenities_count']].copy()
score_df['Price_per_SqFt_inv'] = 1 - sc.fit_transform(score_df[['Price_per_SqFt']])
score_df['PTA_s'] = sc.fit_transform(score_df[['Public_Transport_Accessibility']])
score_df['amen_s'] = sc.fit_transform(score_df[['amenities_count']])
model_df['investment_score'] = (score_df['Price_per_SqFt_inv']*0.5 + score_df['PTA_s']*0.3 + score_df['amen_s']*0.2)
model_df['investment_label'] = (model_df['investment_score'] > model_df['investment_score'].median()).astype(int)
model_df['investment_label'].value_counts()

In [ ]:
# Split for classification
features = ['Size_in_SqFt','Price_per_SqFt','Public_Transport_Accessibility','Nearby_Schools','Nearby_Hospitals','amenities_count'] + cat_cols
X = model_df[features]
y = model_df['investment_label']
# Simple preprocessing pipeline
numeric_features = ['Size_in_SqFt','Price_per_SqFt','Public_Transport_Accessibility','Nearby_Schools','Nearby_Hospitals','amenities_count']
numeric_transformer = Pipeline([('scaler', StandardScaler())])
categorical_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, cat_cols)
])
clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', RandomForestClassifier(n_estimators=50, random_state=42))])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Train quickly
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, pred))
print('F1-score:', f1_score(y_test, pred))
print(classification_report(y_test, pred))
print('Confusion matrix:\n', confusion_matrix(y_test, pred))
# Save classifier
joblib.dump(clf, 'investment_classifier.joblib')

# Log with MLflow (local)
mlflow.set_experiment('real_estate_quick')
with mlflow.start_run(run_name='rf_investment_small'):
    mlflow.log_param('model','RandomForestClassifier')
    mlflow.log_metric('accuracy', accuracy_score(y_test, pred))
    mlflow.log_metric('f1', f1_score(y_test, pred))
    mlflow.sklearn.log_model(clf, 'model')

# Regression: predict Price_in_Lakhs (quick)
reg_features = ['Size_in_SqFt','Price_per_SqFt','Public_Transport_Accessibility','Nearby_Schools','Nearby_Hospitals','amenities_count'] + cat_cols
Xr = model_df[reg_features]
yr = model_df['Price_in_Lakhs']
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.2, random_state=42)
reg_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(n_estimators=50, random_state=42))])
reg_pipe.fit(Xr_train, yr_train)
yr_pred = reg_pipe.predict(Xr_test)
print('RMSE:', np.sqrt(mean_squared_error(yr_test, yr_pred)))
print('MAE:', mean_absolute_error(yr_test, yr_pred))
print('R2:', r2_score(yr_test, yr_pred))
joblib.dump(reg_pipe, 'price_regressor.joblib')
with mlflow.start_run(run_name='rf_price_small'):
    mlflow.log_param('model','RandomForestRegressor')
    mlflow.log_metric('rmse', np.sqrt(mean_squared_error(yr_test, yr_pred)))
    mlflow.log_metric('mae', mean_absolute_error(yr_test, yr_pred))
    mlflow.sklearn.log_model(reg_pipe, 'model')

In [ ]:
# Write a simple Streamlit app to disk for deployment/testing
streamlit_code = r'''
import streamlit as st
import joblib, pandas as pd
st.title('Real Estate Investment Advisor - Demo')
clf = joblib.load('investment_classifier.joblib')
reg = joblib.load('price_regressor.joblib')
# Collect user inputs
size = st.number_input('Size (SqFt)', value=1000)
ppsf = st.number_input('Price per SqFt', value=0.05, format='%.3f')
pta = st.slider('Public Transport Accessibility (1-10)', 1, 10, 5)
schools = st.number_input('Nearby Schools', 0, 50, 1)
hospitals = st.number_input('Nearby Hospitals', 0, 50, 1)
amen = st.number_input('Amenities count', 0, 10, 2)
ptype = st.selectbox('Property Type', ['Apartment','Independent House','Villa'])
furn = st.selectbox('Furnished Status', ['Furnished','Semi-furnished','Unfurnished'])
owner = st.selectbox('Owner Type', ['Owner','Builder','Broker'])
facing = st.selectbox('Facing', ['North','South','East','West'])
if st.button('Predict'):
    X = pd.DataFrame([{
        'Size_in_SqFt': size, 'Price_per_SqFt': ppsf, 'Public_Transport_Accessibility': pta,
        'Nearby_Schools': schools, 'Nearby_Hospitals': hospitals, 'amenities_count': amen,
        'Property_Type': ptype, 'Furnished_Status': furn, 'Owner_Type': owner, 'Facing': facing
    }])
    inv_proba = clf.predict_proba(X)[0,1] if hasattr(clf, 'predict_proba') else clf.predict(X)
    price_pred = reg.predict(X)[0]
    st.write('Investment probability (higher is better):', float(inv_proba))
    st.write('Predicted Price (Lakhs):', float(price_pred))
'''
with open('streamlit_app.py','w', encoding='utf-8') as f:
    f.write(streamlit_code)
print('Wrote streamlit_app.py to disk. Run with: streamlit run streamlit_app.py')

**Next steps / Notes**
- The notebook performs EDA answers for questions asked and trains quick models.
- For production: expand feature engineering, hyperparameter tuning, cross-validation, and add more robust imputation.
- To run Streamlit: in terminal run `streamlit run streamlit_app.py`.
- MLflow runs are tracked locally; run `mlflow ui` to inspect them.

If you want, I can now run the notebook cells or run tests, or refine the investment definition and tune the models further.